<a href="https://colab.research.google.com/github/gadmin7/EventExtraction/blob/main/SentenceLevelBasicEventExtractionRAMS_events.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q evaluate
!pip install -q seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 993.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 532.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import json
import evaluate
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict, ClassLabel,Features
import datasets
import torch
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoTokenizer, DebertaForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data = []
with open(r"/content/drive/MyDrive/NLP_event_extraction/train.jsonlines",encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
data_test = []
with open(r"/content/drive/MyDrive/NLP_event_extraction/dev.jsonlines",encoding='utf-8') as f:
    for line in f:
        data_test.append(json.loads(line))

In [ ]:
df1 = pd.DataFrame(data_test)
df1.head(2)

In [ ]:
df1.shape

In [ ]:
df = pd.DataFrame(data)
df.head(2)

In [ ]:
df1.drop(['rel_triggers','gold_rel_links','language_id','source_url','split'],axis=1,inplace=True)

In [ ]:
df.drop(['rel_triggers','gold_rel_links','language_id','source_url','split'],axis=1,inplace=True)

In [ ]:
%%time
dataframe_list = []
for idx in range(7329):

    row_tokens = []
    trigger_tags = []
    arg_tags = []
    doc_key = []
    sentence_num = []
    for i in range(len(df['sentences'][idx])):
        row_tokens.extend(df['sentences'][idx][i])
        for j in range(len(df['sentences'][idx][i])):
          sentence_num.append(str(i)+df['doc_key'][idx])

    for i in range(len(row_tokens)):
        trigger_tags.append('O')
        arg_tags.append('O')
        doc_key.append(df['doc_key'][idx])

    gold_link = df['gold_evt_links'][idx]

    for i in range(len(gold_link)):

        trigger_list = gold_link[i][0]
        arg_list = gold_link[i][1]
        arg_label = gold_link[i][2]

        trigger_beg = trigger_list[0]
        trigger_end = trigger_list[1]
        trigger_tags[trigger_beg] = 'B-'+ arg_label[:6]
        if trigger_beg != trigger_end:
            for trg in range(trigger_beg+1,trigger_end+1):
                trigger_tags[trg] = 'I-'+ arg_label[:6]

        arg_beg = arg_list[0]
        arg_end = arg_list[1]
        arg_tags[arg_beg] = 'B-'+ arg_label[11:]
        if arg_beg != arg_end:
            for arg in range(arg_beg+1,arg_end+1):
                arg_tags[arg] = 'I-'+ arg_label[11:]

    d1 = pd.DataFrame({'doc_key':sentence_num,'token':row_tokens,'event_tags':trigger_tags,'arg_tags':arg_tags})
    dataframe_list.append(d1)

In [ ]:
%%time
dataframe_test_list = []
for idx in range(df1.shape[0]):

    row_tokens = []
    trigger_tags = []
    arg_tags = []
    doc_key = []
    sentence_num = []
    for i in range(len(df1['sentences'][idx])):
        row_tokens.extend(df1['sentences'][idx][i])
        for j in range(len(df1['sentences'][idx][i])):
          sentence_num.append(str(i)+df1['doc_key'][idx])


    for i in range(len(row_tokens)):
        trigger_tags.append('O')
        arg_tags.append('O')
        doc_key.append(df1['doc_key'][idx])

    gold_link = df1['gold_evt_links'][idx]

    for i in range(len(gold_link)):

        trigger_list = gold_link[i][0]
        arg_list = gold_link[i][1]
        arg_label = gold_link[i][2]

        trigger_beg = trigger_list[0]
        trigger_end = trigger_list[1]
        trigger_tags[trigger_beg] = 'B-'+ arg_label[:6]
        if trigger_beg != trigger_end:
            for trg in range(trigger_beg+1,trigger_end+1):
                trigger_tags[trg] = 'I-'+ arg_label[:6]

        arg_beg = arg_list[0]
        arg_end = arg_list[1]
        arg_tags[arg_beg] = 'B-'+ arg_label[11:]
        if arg_beg != arg_end:
            for arg in range(arg_beg+1,arg_end+1):
                arg_tags[arg] = 'I-'+ arg_label[11:]

    d1 = pd.DataFrame({'doc_key':sentence_num,'token':row_tokens,'event_tags':trigger_tags,'arg_tags':arg_tags})
    dataframe_test_list.append(d1)

In [ ]:
data = pd.concat(dataframe_list)

In [ ]:
test = pd.concat(dataframe_test_list)

In [ ]:
def create_dataset(data):
    grouped_tokens = data.groupby('doc_key')['token'].apply(list).reset_index(name='token')
    grouped_events = data.groupby('doc_key')['event_tags'].apply(list).reset_index(name='event_tags')
    grouped_args = data.groupby('doc_key')['arg_tags'].apply(list).reset_index(name='arg_tags')

    dataset_dict = {
        "id": grouped_tokens['doc_key'],
        "tokens": grouped_tokens['token'],
        "event_tags": grouped_events['event_tags'],
        "arg_tags": grouped_args['arg_tags'],
    }

    return Dataset.from_dict(dataset_dict)

dataset1 = create_dataset(data)

In [ ]:
dataset2 = create_dataset(test)
dataset_dict = DatasetDict({"train": dataset1,"test":dataset2})

In [ ]:
dataset_dict['train'].features

In [ ]:
dataset_dict['test'].features

In [ ]:
event_labels = ['B-evt001', 'B-evt002', 'B-evt003', 'B-evt004', 'B-evt005', 'B-evt006', 'B-evt007', 'B-evt008', 'B-evt009', 'B-evt010', 'B-evt011', 'B-evt012', 'B-evt013', 'B-evt014', 'B-evt015', 'B-evt016', 'B-evt017', 'B-evt018',
                                     'B-evt019', 'B-evt020', 'B-evt021', 'B-evt022', 'B-evt023', 'B-evt024', 'B-evt025', 'B-evt026', 'B-evt027', 'B-evt028', 'B-evt029', 'B-evt030', 'B-evt031', 'B-evt032', 'B-evt033', 'B-evt034', 'B-evt035', 'B-evt036', 'B-evt037', 'B-evt038', 'B-evt039', 'B-evt040', 'B-evt041', 'B-evt042', 'B-evt043', 'B-evt044', 'B-evt045', 'B-evt046', 'B-evt047', 'B-evt048', 'B-evt049', 'B-evt050', 'B-evt051', 'B-evt052', 'B-evt053', 'B-evt054', 'B-evt055', 'B-evt057', 'B-evt059', 'B-evt060', 'B-evt061', 'B-evt062', 'B-evt063', 'B-evt064', 'B-evt065', 'B-evt066', 'B-evt068', 'B-evt070', 'B-evt071', 'B-evt072', 'B-evt073', 'B-evt074', 'B-evt075', 'B-evt076', 'B-evt077', 'B-evt079', 'B-evt080', 'B-evt081', 'B-evt082', 'B-evt083', 'B-evt084', 'B-evt085', 'B-evt086', 'B-evt087', 'B-evt088', 'B-evt089', 'B-evt090', 'B-evt091',
                                     'B-evt092', 'B-evt093', 'B-evt094', 'B-evt095', 'B-evt096', 'B-evt097', 'B-evt098', 'B-evt099', 'B-evt100', 'B-evt101', 'B-evt102', 'B-evt103', 'B-evt104', 'B-evt105', 'B-evt106', 'B-evt107', 'B-evt108', 'B-evt109', 'B-evt110', 'B-evt111', 'B-evt112', 'B-evt113', 'B-evt114', 'B-evt115', 'B-evt116', 'B-evt117', 'B-evt118', 'B-evt119', 'B-evt120', 'B-evt121', 'B-evt122', 'B-evt123', 'B-evt124', 'B-evt125', 'B-evt126', 'B-evt127', 'B-evt128', 'B-evt129', 'B-evt130', 'B-evt131', 'B-evt132', 'B-evt133', 'B-evt134', 'B-evt135', 'B-evt136', 'B-evt137', 'B-evt138', 'B-evt139', 'B-evt140', 'B-evt141', 'B-evt142', 'B-evt143', 'B-evt144', 'I-evt064', 'I-evt065', 'I-evt068', 'I-evt071', 'I-evt073', 'I-evt085', 'I-evt087', 'I-evt089', 'I-evt091', 'I-evt095', 'I-evt100', 'I-evt104', 'I-evt107', 'I-evt112', 'I-evt119', 'I-evt123', 'I-evt124', 'I-evt130', 'I-evt132', 'I-evt133', 'I-evt136', 'I-evt139', 'I-evt142', 'I-evt144', 'O']

In [ ]:
arg_labels = ['B-artifact', 'B-artifactmoney', 'B-attacker', 'B-ballot', 'B-beneficiary', 'B-candidate', 'B-communicator', 'B-crashobject', 'B-crime', 'B-damager', 'B-damagerdestroyer', 'B-deceased', 'B-defendant', 'B-demonstrator', 'B-destination', 'B-destroyer', 'B-detainee', 'B-driverpassenger', 'B-employee',
                                   'B-executioner', 'B-extraditer', 'B-fireexplosionobject', 'B-founder', 'B-giver', 'B-governmentbody', 'B-gpe', 'B-granter', 'B-hidingplace', 'B-injurer', 'B-inspectedentity', 'B-inspector', 'B-instrument', 'B-investigator', 'B-jailer', 'B-judgecourt', 'B-killer', 'B-law', 'B-manufacturer', 'B-money', 'B-monitor', 'B-monitoredentity', 'B-observedentity', 'B-observer', 'B-origin', 'B-otherparticipant', 'B-participant', 'B-passenger', 'B-place', 'B-placeofemployment', 'B-preventer', 'B-prosecutor', 'B-recipient', 'B-rejecternullifier', 'B-result', 'B-retreater', 'B-spy', 'B-surrenderer', 'B-target', 'B-territoryorfacility', 'B-transporter', 'B-vehicle', 'B-victim', 'B-violator',
                                   'B-voter', 'B-yielder', 'I-artifact', 'I-artifactmoney', 'I-attacker', 'I-ballot', 'I-beneficiary', 'I-candidate', 'I-communicator', 'I-crashobject', 'I-crime', 'I-damager', 'I-damagerdestroyer', 'I-deceased', 'I-defendant', 'I-demonstrator', 'I-destination', 'I-destroyer', 'I-detainee', 'I-driverpassenger', 'I-employee', 'I-executioner', 'I-extraditer', 'I-fireexplosionobject', 'I-founder', 'I-giver', 'I-governmentbody', 'I-gpe', 'I-granter', 'I-hidingplace', 'I-injurer', 'I-inspectedentity', 'I-inspector', 'I-instrument', 'I-investigator', 'I-jailer', 'I-judgecourt', 'I-killer', 'I-law', 'I-manufacturer', 'I-money', 'I-monitor', 'I-monitoredentity', 'I-observedentity', 'I-observer', 'I-origin',
                                   'I-otherparticipant', 'I-participant', 'I-passenger', 'I-place', 'I-placeofemployment', 'I-preventer', 'I-prosecutor', 'I-recipient', 'I-rejecternullifier', 'I-result', 'I-retreater', 'I-spy', 'I-surrenderer', 'I-target', 'I-territoryorfacility',
                                   'I-transporter', 'I-vehicle', 'I-victim', 'I-violator', 'I-voter', 'I-yielder', 'O']

In [ ]:
dataset_dict = dataset_dict.cast_column("event_tags", datasets.Sequence(ClassLabel(names=event_labels)))
dataset_dict = dataset_dict.cast_column("arg_tags", datasets.Sequence(ClassLabel(names=arg_labels)))

In [ ]:
dataset_dict['test'].features

In [ ]:
label_list = dataset_dict["train"].features["event_tags"].feature.names

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base",max_length=512,padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
example = dataset_dict["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'transportation',
 'officials',
 'are',
 'urging',
 'carp',
 '##ool',
 'and',
 'tel',
 '##ew',
 '##or',
 '##king',
 'as',
 'options',
 'to',
 'combat',
 'an',
 'expected',
 'flood',
 'of',
 'drivers',
 'on',
 'the',
 'road',
 '.',
 '[SEP]']

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True,max_length=512)

    labels = []
    for i, label in enumerate(examples[f"event_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_rams = dataset_dict.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/35535 [00:00<?, ? examples/s]

Map:   0%|          | 0/4456 [00:00<?, ? examples/s]

In [ ]:
tokenized_rams

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'event_tags', 'arg_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 35535
    })
    test: Dataset({
        features: ['id', 'tokens', 'event_tags', 'arg_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4456
    })
})

In [ ]:
for i in range(7329):
  if len(tokenized_rams['train'][i]['input_ids']) > 500:
    print(len(tokenized_rams['train'][i]['input_ids']),i)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
seqeval = evaluate.load("seqeval")

In [ ]:
tokenized_rams

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'event_tags', 'arg_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 35535
    })
    test: Dataset({
        features: ['id', 'tokens', 'event_tags', 'arg_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4456
    })
})

In [ ]:
labels = [label_list[i] for i in example["event_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels,zero_division=1)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
encoder = LabelEncoder()
data['evt_tags_en'] = encoder.fit_transform(data['event_tags'])

label2id = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
id2label = dict(zip(encoder.transform(encoder.classes_),encoder.classes_))

In [ ]:
id2label = {0: 'B-evt001',1: 'B-evt002',2: 'B-evt003',3: 'B-evt004',4: 'B-evt005',5: 'B-evt006',6: 'B-evt007',7: 'B-evt008',8: 'B-evt009',9: 'B-evt010',10: 'B-evt011',11: 'B-evt012',
12: 'B-evt013',13: 'B-evt014',14: 'B-evt015',15: 'B-evt016',16: 'B-evt017',17: 'B-evt018',18: 'B-evt019',19: 'B-evt020',20: 'B-evt021',21: 'B-evt022',22: 'B-evt023',23: 'B-evt024',
24: 'B-evt025',25: 'B-evt026',26: 'B-evt027',27: 'B-evt028',28: 'B-evt029',29: 'B-evt030',30: 'B-evt031', 31: 'B-evt032',32: 'B-evt033',33: 'B-evt034',34: 'B-evt035',
35: 'B-evt036',36: 'B-evt037',37: 'B-evt038',38: 'B-evt039',39: 'B-evt040',40: 'B-evt041',41: 'B-evt042',42: 'B-evt043',43: 'B-evt044',44: 'B-evt045',45: 'B-evt046',46: 'B-evt047',
47: 'B-evt048',48: 'B-evt049',49: 'B-evt050',50: 'B-evt051',51: 'B-evt052',52: 'B-evt053',53: 'B-evt054',54: 'B-evt055',55: 'B-evt057',56: 'B-evt059',57: 'B-evt060',58: 'B-evt061',
59: 'B-evt062',60: 'B-evt063',61: 'B-evt064',62: 'B-evt065',63: 'B-evt066',64: 'B-evt068',65: 'B-evt070',66: 'B-evt071',67: 'B-evt072',68: 'B-evt073',69: 'B-evt074',70: 'B-evt075',
71: 'B-evt076',72: 'B-evt077',73: 'B-evt079',74: 'B-evt080',75: 'B-evt081',76: 'B-evt082',77: 'B-evt083',78: 'B-evt084',79: 'B-evt085',80: 'B-evt086',81: 'B-evt087',82: 'B-evt088',
83: 'B-evt089',84: 'B-evt090',85: 'B-evt091',86: 'B-evt092',87: 'B-evt093',88: 'B-evt094',89: 'B-evt095',90: 'B-evt096',91: 'B-evt097',92: 'B-evt098',93: 'B-evt099',94: 'B-evt100',
95: 'B-evt101',96: 'B-evt102',97: 'B-evt103',98: 'B-evt104',99: 'B-evt105',100: 'B-evt106',101: 'B-evt107',102: 'B-evt108',103: 'B-evt109',104: 'B-evt110',105: 'B-evt111',
106: 'B-evt112',107: 'B-evt113',108: 'B-evt114', 109: 'B-evt115',110: 'B-evt116',111: 'B-evt117',112: 'B-evt118',113: 'B-evt119',114: 'B-evt120',115: 'B-evt121',116: 'B-evt122',
117: 'B-evt123',118: 'B-evt124',119: 'B-evt125',120: 'B-evt126',121: 'B-evt127',122: 'B-evt128',123: 'B-evt129',124: 'B-evt130',125: 'B-evt131',126: 'B-evt132',127: 'B-evt133',
128: 'B-evt134',129: 'B-evt135',130: 'B-evt136',131: 'B-evt137',132: 'B-evt138',133: 'B-evt139',134: 'B-evt140',135: 'B-evt141',136: 'B-evt142',137: 'B-evt143',138: 'B-evt144',139: 'I-evt064',
140: 'I-evt065',141: 'I-evt068',142: 'I-evt071',143: 'I-evt073',144: 'I-evt085',145: 'I-evt087',146: 'I-evt089',147: 'I-evt091',148: 'I-evt095',149: 'I-evt100',150: 'I-evt104',151: 'I-evt107',
152: 'I-evt112',153: 'I-evt119',154: 'I-evt123',155: 'I-evt124',156: 'I-evt130',157: 'I-evt132',158: 'I-evt133',159: 'I-evt136',160: 'I-evt139',161: 'I-evt142',162: 'I-evt144',
163: 'O'}

In [ ]:
label2id = {'B-evt001': 0,'B-evt002': 1,'B-evt003': 2,'B-evt004': 3,'B-evt005': 4,'B-evt006': 5,'B-evt007': 6,'B-evt008': 7,'B-evt009': 8,'B-evt010': 9,'B-evt011': 10,'B-evt012': 11,
 'B-evt013': 12,'B-evt014': 13,'B-evt015': 14,'B-evt016': 15,'B-evt017': 16,'B-evt018': 17,'B-evt019': 18,'B-evt020': 19,'B-evt021': 20,'B-evt022': 21,'B-evt023': 22,'B-evt024': 23,
 'B-evt025': 24,'B-evt026': 25,'B-evt027': 26,'B-evt028': 27,'B-evt029': 28,'B-evt030': 29,'B-evt031': 30,'B-evt032': 31,'B-evt033': 32,'B-evt034': 33,'B-evt035': 34,'B-evt036': 35,
 'B-evt037': 36,'B-evt038': 37,'B-evt039': 38,'B-evt040': 39,'B-evt041': 40,'B-evt042': 41,'B-evt043': 42,'B-evt044': 43,'B-evt045': 44,'B-evt046': 45,'B-evt047': 46,'B-evt048': 47,
 'B-evt049': 48,'B-evt050': 49,'B-evt051': 50,'B-evt052': 51,'B-evt053': 52,'B-evt054': 53,'B-evt055': 54,'B-evt057': 55,'B-evt059': 56,'B-evt060': 57,'B-evt061': 58,'B-evt062': 59,
 'B-evt063': 60, 'B-evt064': 61,'B-evt065': 62,'B-evt066': 63,'B-evt068': 64,'B-evt070': 65,'B-evt071': 66,'B-evt072': 67,'B-evt073': 68,'B-evt074': 69,'B-evt075': 70,'B-evt076': 71,
 'B-evt077': 72,'B-evt079': 73,'B-evt080': 74,'B-evt081': 75,'B-evt082': 76,'B-evt083': 77,'B-evt084': 78,'B-evt085': 79,'B-evt086': 80,'B-evt087': 81,'B-evt088': 82,'B-evt089': 83,
 'B-evt090': 84,'B-evt091': 85,'B-evt092': 86,'B-evt093': 87,'B-evt094': 88,'B-evt095': 89,'B-evt096': 90,'B-evt097': 91,'B-evt098': 92,'B-evt099': 93,'B-evt100': 94,'B-evt101': 95,
 'B-evt102': 96,'B-evt103': 97,'B-evt104': 98,'B-evt105': 99,'B-evt106': 100,'B-evt107': 101,'B-evt108': 102,'B-evt109': 103,'B-evt110': 104,'B-evt111': 105,'B-evt112': 106,'B-evt113': 107,
 'B-evt114': 108,'B-evt115': 109,'B-evt116': 110,'B-evt117': 111,'B-evt118': 112,'B-evt119': 113,'B-evt120': 114,'B-evt121': 115,'B-evt122': 116,'B-evt123': 117,'B-evt124': 118,'B-evt125': 119,
 'B-evt126': 120,'B-evt127': 121,'B-evt128': 122,'B-evt129': 123,'B-evt130': 124,'B-evt131': 125,'B-evt132': 126,'B-evt133': 127,'B-evt134': 128,'B-evt135': 129,'B-evt136': 130,'B-evt137': 131,
 'B-evt138': 132,'B-evt139': 133,'B-evt140': 134,'B-evt141': 135,'B-evt142': 136,'B-evt143': 137,'B-evt144': 138,'I-evt064': 139,'I-evt065': 140,'I-evt068': 141,'I-evt071': 142,'I-evt073': 143,
 'I-evt085': 144,'I-evt087': 145,'I-evt089': 146,'I-evt091': 147,'I-evt095': 148,'I-evt100': 149,'I-evt104': 150,'I-evt107': 151,'I-evt112': 152,'I-evt119': 153,'I-evt123': 154,'I-evt124': 155,
 'I-evt130': 156,'I-evt132': 157,'I-evt133': 158,'I-evt136': 159,'I-evt139': 160,'I-evt142': 161,'I-evt144': 162,'O': 163}

In [ ]:
model = DebertaForTokenClassification.from_pretrained("microsoft/deberta-base").to(device)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/NLP_event_extraction/sentence_level_ee_rams_event_model",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=7,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_rams["train"],
    eval_dataset=tokenized_rams["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.036300,0.033386,0.294461,0.112723,0.163035,0.992065
2,0.029900,0.032410,0.276888,0.135045,0.181545,0.991890
3,0.025600,0.033918,0.276243,0.167411,0.208478,0.991664


In [ ]:
!pip install accelerate -U

In [ ]:
eg = []
with open(r"/content/drive/MyDrive/NLP_event_extraction/test.jsonlines",encoding='utf-8') as f:
    for line in f:
        eg.append(json.loads(line))

In [ ]:
dtest = pd.DataFrame(eg)
dtest.head(2)

,rel_triggers,gold_rel_links,doc_key,ent_spans,language_id,source_url,evt_triggers,split,sentences,gold_evt_links
0,[],[],nw_RC00c8620ef5810429342a1c339e6c76c1b0b9add3f...,"[[27, 27, [['evt043arg01communicator', 1.0]]],...",eng,http://bbc.co.uk/sport/athletics/36295481,"[[31, 31, [['contact.prevarication.broadcast',...",test,"[[We, are, ashamed, of, them, ., ""], [However,...","[[[31, 31], [27, 27], evt043arg01communicator]..."
1,[],[],nw_RC013c8e78b7e8a4fb22193483877058f712dfd8b75...,"[[37, 37, [['evt090arg02victim', 1.0]]]]",eng,https://www.rt.com/news/333928-woman-severed-h...,"[[40, 40, [['life.die.deathcausedbyviolenteven...",test,"[[I, thought, ,, wow, ,, it, must, be, a, repl...","[[[40, 40], [37, 37], evt090arg02victim]]"


In [ ]:
print(dtest['sentences'][0])

[['We', 'are', 'ashamed', 'of', 'them', '.', '"'], ['However', ',', 'Mutko', 'stopped', 'short', 'of', 'admitting', 'the', 'doping', 'scandal', 'was', 'state', 'sponsored', '.'], ['"', 'We', 'are', 'very', 'sorry', 'that', 'athletes', 'who', 'tried', 'to', 'deceive', 'us', ',', 'and', 'the', 'world', ',', 'were', 'not', 'caught', 'sooner', '.'], ['We', 'are', 'very', 'sorry', 'because', 'Russia', 'is', 'committed', 'to', 'upholding', 'the', 'highest', 'standards', 'in', 'sport', 'and', 'is', 'opposed', 'to', 'anything', 'that', 'threatens', 'the', 'Olympic', 'values', ',', '"', 'he', 'said', '.'], ['English', 'former', 'heptathlete', 'and', 'Athens', '2004', 'bronze', 'medallist', 'Kelly', 'Sotherton', 'was', 'unhappy', 'with', 'Mutko', "'s", 'plea', 'for', 'Russia', "'s", 'ban', 'to', 'be', 'lifted', 'for', 'Rio']]


In [ ]:
print(dtest['gold_evt_links'][0],dtest['evt_triggers'][0])

[[[31, 31], [27, 27], 'evt043arg01communicator'], [[31, 31], [32, 36], 'evt043arg02recipient'], [[31, 31], [48, 48], 'evt043arg03place']] [[31, 31, [['contact.prevarication.broadcast', 1.0]]]]


In [ ]:
%%time
dataframe_list = []
for idx in range(871):

    row_tokens = []
    trigger_tags = []
    arg_tags = []
    doc_key = []
    sentence_num = []
    for i in range(len(dtest['sentences'][idx])):
        row_tokens.extend(dtest['sentences'][idx][i])
        for j in range(len(dtest['sentences'][idx][i])):
          sentence_num.append(str(i)+dtest['doc_key'][idx])

    for i in range(len(row_tokens)):
        trigger_tags.append('O')
        arg_tags.append('O')
        doc_key.append(dtest['doc_key'][idx])

    gold_link = dtest['gold_evt_links'][idx]

    for i in range(len(gold_link)):

        trigger_list = gold_link[i][0]
        arg_list = gold_link[i][1]
        arg_label = gold_link[i][2]

        trigger_beg = trigger_list[0]
        trigger_end = trigger_list[1]
        trigger_tags[trigger_beg] = 'B-'+ arg_label[:6]
        if trigger_beg != trigger_end:
            for trg in range(trigger_beg+1,trigger_end+1):
                trigger_tags[trg] = 'I-'+ arg_label[:6]

        arg_beg = arg_list[0]
        arg_end = arg_list[1]
        arg_tags[arg_beg] = 'B-'+ arg_label[11:]
        if arg_beg != arg_end:
            for arg in range(arg_beg+1,arg_end+1):
                arg_tags[arg] = 'I-'+ arg_label[11:]

    d1 = pd.DataFrame({'doc_key':sentence_num,'token':row_tokens,'event_tags':trigger_tags,'arg_tags':arg_tags})
    dataframe_list.append(d1)

CPU times: user 1.53 s, sys: 23.2 ms, total: 1.55 s
Wall time: 1.54 s


In [ ]:
eg_test = pd.concat(dataframe_list)

In [ ]:
def create_dataset(data):
    grouped_tokens = data.groupby('doc_key')['token'].apply(list).reset_index(name='token')
    grouped_events = data.groupby('doc_key')['event_tags'].apply(list).reset_index(name='event_tags')
    grouped_args = data.groupby('doc_key')['arg_tags'].apply(list).reset_index(name='arg_tags')

    dataset_dict = {
        "id": grouped_tokens['doc_key'],
        "tokens": grouped_tokens['token'],
        "event_tags": grouped_events['event_tags'],
        "arg_tags": grouped_args['arg_tags'],
    }

    return Dataset.from_dict(dataset_dict)

dataset1 = create_dataset(eg_test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP_event_extraction/sentence_level_ee_rams_event_model/checkpoint-106605",max_length=512,padding=True)

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/NLP_event_extraction/sentence_level_ee_rams_event_model/checkpoint-106605")

In [ ]:
def all_same(items):
    return all(x == items[0] for x in items)

for i in range(1000):
  idx = i
  c = dataset1[idx]['tokens']
  b = dataset1[idx]['event_tags']
  if all_same(b):
    pass
  else:
    text = ' '.join(c)
    print(i)
    print(text)

    inputs = tokenizer(text,return_tensors="pt")
    # inputs = tokenize_and_align_labels(dataset1[idx])
    with torch.no_grad():
        logits = model(**inputs).logits


    predictions = torch.argmax(logits, dim=2)
    predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
    print(predicted_token_class)
    print(b)

25
Russia has called for a thorough and unbiased investigation into a Saudi airstrike that killed over 140 people at a funeral in the Yemeni capital city of Sana’a , urging punitive measures against all those responsible for the attack .
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-evt035', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
34
Richard L. Campbell , the man accused of sucker - punching a 69-year - old protester outside a Donald Trump rally in Asheville , NC on Monday , is vigorously denying the charge — and has video that his attorney says proves his innocence .
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 